# Creating Portfolio based on Risk Apetite

In [1]:
pip install yfinance

Note: you may need to restart the kernel to use updated packages.


# MAIN FUNCTION

In [103]:
# Main executing function
import pandas as pd
import numpy as np
import yfinance as yf
import datetime
from IPython.display import clear_output


print("Welcome! We are your Portfolio Manager.\nKindly answer a few questions to help us know you.")

# Define options for investment objective, source of income, and expected return
investment_objective_options = {
    '1': 'Retirement savings',
    '2': 'Wealth preservation',
    '3': 'Wealth accumulation'
}

source_of_income_options = {
    '1': 'Salary',
    '2': 'Debt',
    '3': 'Previous Investment'
}

time_horizon_options = {
    '1': 'Less than 5 years',
    '2': 'Between 5-10 years',
    '3': 'More than 10 years'
}

# Function to ask a question with options
def ask_question_with_options(question, options):
    print(question)
    for key, value in options.items():
        print(f"{key}. {value}")
    choice = input("Enter the number corresponding to your choice: ")
    return options.get(choice)

# Ask investment objective
investment_objective = ask_question_with_options("Q. What is your primary investment objective?", investment_objective_options)

# Ask source of income
source_of_income = ask_question_with_options("\nQ. What is your primary source of income?", source_of_income_options)

# Ask expected return
time_horizon = ask_question_with_options("\nQ. For how long do you wish to invest?", time_horizon_options)

clear_output(wait=True)  #Clear previous output

print("\nThank you for completing the questionnaire!")
print("\n\nSummary of Your Responses:")
print("\nInvestment Objective:", investment_objective)
print("\nSource of Investment:", source_of_income)
print("\nTime Horizon:", time_horizon)

print("\nPlease wait, while we analyze your portfolio.")


user_risk = categorize_risk_appetite(investment_objective, source_of_income, time_horizon)
print("User's Risk Appetite:", user_risk)


ticker_symbol = ['RELIANCE.NS','DMART.NS','ADANIPOWER.NS','INFY.NS','TCS.NS','OBEROIRLTY.NS','MRPL.NS','LTTS.NS','HINDPETRO.NS','JYOTHYLAB.NS','SDBL.NS','COALINDIA.NS','VADILALIND.NS','GREENPOWER.NS','NDTV.NS']
#company data required

risk_scores_df=pd.DataFrame()
Beta_df = pd.DataFrame(columns=['Company', 'Beta Value'])

#loop for iterating through each stock
for ticker in ticker_symbol:
   #calling function to fetch latest data from Yahoo finance 
   company_info, df_historical_prices, df_income_statement, df_balance_sheet=fetching_data(ticker) 
   
   #Fundamental analysis on the returned values
   sector_name,Beta, DE_Ratio, interest_coverage_ratio,pe_ratio, pb_ratio, latest_eps, ROE, ROCE, market_cap = fundamental_analysis(company_info, df_income_statement, df_historical_prices,df_balance_sheet)
   
   Beta_df=pd.concat([Beta_df,pd.DataFrame({'Company':[ticker], 'Beta': [Beta]})], ignore_index=True
                    )
    #technical analysis to get the latest buy or sell signal
   rsi_signal, latest_macd_signal, mean_return, std_dev,mean_return = technical_analysis(df_historical_prices)

   #calculating risk score by creating bands and normalising different metrics to get a comparable score
   score=risk_score(sector_name,DE_Ratio, interest_coverage_ratio, pe_ratio, pb_ratio, ROE, ROCE, rsi_signal, latest_macd_signal, mean_return, std_dev)
   
    #storing the risk scores in a dataframe corresponding to the company
   risk_scores_df = pd.concat([risk_scores_df, pd.DataFrame({'Company': [ticker], 'Market Cap': [market_cap], 'Risk Score': [score]})], ignore_index=True)


portfolio_df = portfolio(risk_scores_df, user_risk)
portfolio_df=pd.DataFrame(portfolio_df)

print("Here are your Portfolio Details:\n")
print("{} portfolio:\n{}".format(user_risk, portfolio_df))

portfolio_return, portfolio_risk, sharpe_ratio, treynor_ratio =risk_return(portfolio_df,Beta_df) 

#final portfolio risk and return
print("The return for the portfolio is: ", portfolio_return)
print("The risk for the portfolio is: ", portfolio_risk)

#printing portfolio measures
print("\n Sharpe Ratio: ", sharpe_ratio)
print("Treynor's Ratio: ", treynor_ratio)
   


Thank you for completing the questionnaire!


Summary of Your Responses:

Investment Objective: Wealth preservation

Source of Income: Previous Investment

Time Horizon: Between 5-10 years

Please wait, while we analyze your portfolio.
User's Risk Appetite: Medium Risk Appetite


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%*******

Here are your Portfolio Details:

Medium Risk Appetite portfolio:
          Company      Market Cap  Risk Score    Weight
0     RELIANCE.NS  19834851033088         195  0.149877
13  GREENPOWER.NS     20938457088         195  0.149877
14        NDTV.NS     14115990528         195  0.149877
7         LTTS.NS    562283413504         205  0.142566
5   OBEROIRLTY.NS    531095257088         215  0.135935
6         MRPL.NS    395123916800         215  0.135935
11   COALINDIA.NS   2792949284864         215  0.135935
The return for the portfolio is:  0.3786549830738201
The risk for the portfolio is:  0.4415933514436413

 Sharpe Ratio:  0.6951078001295395
Treynor's Ratio:  0.47269574039708134


# FETCHING DATA

In [8]:
# ticker symbol of the company

def fetching_data(ticker): 
    company_data = yf.Ticker(ticker)
    start_date=datetime.datetime(2019,4,7)
    end_date=datetime.datetime(2024,4,7)

    #Retrieve company information
    company_info = dict(company_data.info)

    #Retrieve historical stock prices
    historical_prices = company_data.history(start=start_date, end=end_date)
    df_historical_prices= pd.DataFrame(historical_prices)

    #Retrieve financial ratios
    income_statement = company_data.financials
    df_income_statement = pd.DataFrame(income_statement)

    #Retrieve balance sheet
    balance_sheet = company_data.balance_sheet
    df_balance_sheet = pd.DataFrame(balance_sheet)
   
    #returning created dataframes to be used in other functions
    return company_info, df_historical_prices, df_income_statement, df_balance_sheet



# FUNDAMENTAL ANALYSIS

In [7]:
def fundamental_analysis(company_info, df_income_statement, df_historical_prices, df_balance_sheet):
    #Retrieving the Industry name of the company
    sector_name=company_info['sector']
   
    #Retrieving market cap
    market_cap=company_info['marketCap']

   #retrieving beta value
    Beta=company_info['beta']
    
   #Leverage Ratios

   #DE_Ratio=company_info['debtToEquity']
    DE_Ratio = df_balance_sheet.loc['Total Debt'].iloc[0]/df_balance_sheet.loc['Stockholders Equity'].iloc[0]
 
    #interest coverage ratio
    interest_coverage_ratio= df_income_statement.loc['EBIT'].iloc[0]/df_income_statement.loc['Interest Expense'].iloc[0] 
 
   #P/E ratio
    pe_ratio= df_historical_prices['Close'].iloc[-1]/df_income_statement.loc['Basic EPS'].iloc[0]
  
   #P/B Ratio
    pb_ratio = company_info['priceToBook']

   #Earnings Per Share
    latest_eps=df_income_statement.loc['Basic EPS'].iloc[0]

   #PROFITABILITY RATIO
   
   #Return On Equity
    ROE= df_income_statement.loc['Net Income'].iloc[0]/df_balance_sheet.loc['Stockholders Equity'].iloc[0]
   
   #Return on Capital Employed

    CL=df_balance_sheet[12:66].loc['Current Liabilities'].iloc[0]
    TA=df_balance_sheet[12:66].loc['Total Assets'].iloc[0]
    Capital_Employed=TA-CL
    ROCE=df_income_statement.loc['EBIT'].iloc[0]/Capital_Employed

    return sector_name, Beta, DE_Ratio, interest_coverage_ratio,pe_ratio, pb_ratio, latest_eps, ROE, ROCE, market_cap 




    

# TECHNICAL ANALYSIS

In [59]:
def technical_analysis(df_historical_prices):
    data = df_historical_prices
    window = 14
    
    # Calculate daily returns
    df_historical_prices['return'] = df_historical_prices['Close'].pct_change()

    # Calculate mean return (average daily return)
    mean_return = df_historical_prices['return'].mean()
#    print("Mean return of stock: ",mean_return)
    
   
    std_dev = df_historical_prices['return'].std()
#    print("Std of stock: ",std_dev)

    def calculate_rsi(data, window):
        delta = data['Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()
        rs = gain / loss
        rsi_values = 100 - (100 / (1 + rs))
        return rsi_values

    rsi_values = calculate_rsi(data, window)
    # Identify overbought and oversold conditions
    #Buy signal is 1 and sell signal is -1 and Hold is 0
    latest_rsi_value = rsi_values[-1]  # Get the last RSI value
    if latest_rsi_value > 70:
        rsi_signal = 0  # Sell signal
    elif latest_rsi_value < 30:
        rsi_signal = -1  # Buy signal
    else:
        rsi_signal = 0

    # Define the calculate_macd function
    def calculate_macd(data, short_window=12, long_window=26, signal_window=9):
        short_ema = data['Close'].ewm(span=short_window, min_periods=1, adjust=False).mean()
        long_ema = data['Close'].ewm(span=long_window, min_periods=1, adjust=False).mean()
        macd_line = short_ema - long_ema
        signal_line = macd_line.ewm(span=signal_window, min_periods=1, adjust=False).mean()
        histogram = macd_line - signal_line
        return macd_line, signal_line, histogram
    

    # Define buy and sell signals function
    # 1 is buy, -1 is sell, 0 is hold
    def macd(macd_line, signal_line):
        macd_signals = []
        prev_signal = None
        for macd_val, signal_val in zip(macd_line, signal_line):
            if macd_val > signal_val:
                if prev_signal != 1:
                    macd_signals.append(1)
                    prev_signal = 1
            elif macd_val < signal_val:
                if prev_signal != -1:
                    macd_signals.append(-1)
                    prev_signal = -1
            else:
                macd_signals.append(0)
        return macd_signals
    
    # Calculate MACD
    macd_line, signal_line, histogram = calculate_macd(df_historical_prices)

    # Generate buy and sell signals
    macd_signal = macd(macd_line, signal_line)
#    print("Rsi signal for stock: ", rsi_signal)
    
    latest_macd_signal = macd_signal[-1]  # Get the last MACD signal
#    print("latest macd signal",latest_macd_signal)
    return rsi_signal, latest_macd_signal, mean_return, std_dev, mean_return


# RISK SCORE 

In [61]:

def risk_score(sector_name, DE_Ratio, interest_coverage_ratio, pe_ratio,
               pb_ratio, ROE, ROCE, rsi_signal, macd_signal, mean_return, std_dev):
    score_band = [10, 20, 30]
    score = 0
    sector_to_ticker = {
        "energy": "^CNXENERGY",
        "consumer cyclical": "^CNXAUTO",
        'financial services': "NIFTY_FIN_SERVICE.NS",
        'healthcare': "NIFTY_HEALTHCARE.NS",
        'utilities': "^CNXCMDT",
        'technology': "^CNXIT",
        'real estate': "^CNXREALTY",
        'communication services': "^CNXIT",
        'industrials': "^CNXENERGY",
        'consumer defensive': "^CNXFMCG"
    }

    # Get the ticker code corresponding to the sector name
    ticker_code = sector_to_ticker.get(sector_name.lower(), None)

    if ticker_code:
        # Load sector data
        sec_historical_prices = yf.download(ticker_code, period="5y")

        sec_historical_prices['return'] = sec_historical_prices['Close'].pct_change()
        sec_return = (sec_historical_prices['return'].mean())
    #   print("sc_return:", sec_return)
        sec_dev = (sec_historical_prices['return'].std())
     #  print("sec_dev", sec_dev)

        # Define condition ranges for each parameter based on industry
        industry_conditions = {
            "energy": {
                "DE_Ratio": [0.5, 1],
                "interest_coverage_ratio": [5,20],
                "pe_ratio": [10,25],
                "pb_ratio": [1.65, 2.47],
                "ROE": [0.10,0.30],
                "ROCE": [0.10,0.20],
                "mean_return": None,  # No specific range, to be calculated later
                "std_dev": None  # No specific range, to be calculated later
            },
            "consumer cyclicals": {
                "DE_Ratio": [0.1, 0.5],
                "interest_coverage_ratio": [20, 60],
                "pe_ratio": [20, 60],
                "pb_ratio": [2, 8],
                "ROE": [0.10,0.20],
                "ROCE": [0.10, 0.25],
                "mean_return": None,  # No specific range, to be calculated later
                "std_dev": None  # No specific range, to be calculated later
            },
            "financial services": {
               "DE_Ratio": [5,13],
               "interest_coverage_ratio": [1, 2],
                "pe_ratio": [12, 25],
                "pb_ratio": [2, 6],
                "ROE": [0.10, 0.40],
                "ROCE": [0.03,0.10],
                "mean_return": None,  # No specific range, to be calculated later
                "std_dev": None  # No specific range, to be calculated later   
            },
            "healthcare": {
               "DE_Ratio": [0.1, 0.5],
               "interest_coverage_ratio": [20, 40],
                "pe_ratio": [35, 60],
                "pb_ratio": [3, 10],
                "ROE": [0.09,0.25],
                "ROCE": [0.10,0.25],
                "mean_return": None,  # No specific range, to be calculated later
                "std_dev": None  # No specific range, to be calculated later   
            },               
            "utilities":{
               "DE_Ratio": [0.2, 1.2],
               "interest_coverage_ratio": [2, 10],
                "pe_ratio": [20, 60],
                "pb_ratio": [0, 6],
                "ROE": [0.05, 0.25],
                "ROCE": [0.07, 0.20],
                "mean_return": None,  # No specific range, to be calculated later
                "std_dev": None  # No specific range, to be calculated later   
            },
            "technology":{
               "DE_Ratio": [0, 0.5],
               "interest_coverage_ratio": [20, 55],
                "pe_ratio": [15,35],
                "pb_ratio": [5,15],
                "ROE": [0.10, 0.35],
                "ROCE": [0.15,0.35],
                "mean_return": None,  # No specific range, to be calculated later
                "std_dev": None  # No specific range, to be calculated later   
            }, 
            "real estate":{
               "DE_Ratio": [0.2, 1.5],
               "interest_coverage_ratio": [2, 10],
                "pe_ratio": [40, 100],
                "pb_ratio": [2, 9],
                "ROE": [0.03, 0.09],
                "ROCE": [0.04, 0.09],
                "mean_return": None,  # No specific range, to be calculated later
                "std_dev": None  # No specific range, to be calculated later   
            },
            "communication services":{
               "DE_Ratio": [0, 0.5],
               "interest_coverage_ratio": [20, 55],
                "pe_ratio": [15, 35],
                "pb_ratio": [5, 15],
                "ROE": [0.10, 0.35],
                "ROCE": [0.15,0.35],
                "mean_return": None,  # No specific range, to be calculated later
                "std_dev": None  # No specific range, to be calculated later   
            },
            "industrials":{
               "DE_Ratio": [0.5, 1],
               "interest_coverage_ratio": [5, 20],
                "pe_ratio": [10,25],
                "pb_ratio": [1.65, 2.47],
                "ROE": [0.10,0.30],
                "ROCE": [0.10,0.20],
                "mean_return": None,  # No specific range, to be calculated later
                "std_dev": None  # No specific range, to be calculated later   
            },   
            "consumer defensive":{
               "DE_Ratio": [0, 0.6],
               "interest_coverage_ratio": [100, 160],
                "pe_ratio": [50, 80],
                "pb_ratio": [15, 35],
                "ROE": [0.20, 0.50],
                "ROCE": [0.30, 0.60],
                "mean_return": None,  # No specific range, to be calculated later
                "std_dev": None  # No specific range, to be calculated later   
            }
            }
            

        conditions = industry_conditions.get(sector_name.lower(), {})

        for parameter, bounds in conditions.items():
            value = locals()[parameter]  # Get the value of the parameter variable
            if bounds:
                if bounds[0] < value < bounds[1]:
                    score += score_band[1]
                elif value < bounds[0]:
                    score += score_band[0]
                else:
                    score += score_band[2]
      #          print("score for the metric: ",score)

        # Calculate risk score for mean_return based on sec_return percentiles
        if "mean_return" in conditions:
            lower_bound_sec =np.percentile(sec_return, 33)
            upper_bound_sec = np.percentile(sec_return, 66)
            if mean_return < lower_bound_sec:
                score += score_band[0]
            elif lower_bound_sec < mean_return < upper_bound_sec:
                score += score_band[1]
            else:
                score += score_band[2]
    #        print("score after return: ", score)
        # Calculate risk score for std_dev based on sec_dev percentiles
        if "std_dev" in conditions:
            lower_bound_dev = np.percentile(sec_return, 33)
            upper_bound_dev = np.percentile(sec_return, 66)
            if std_dev < lower_bound_dev:
                score += score_band[0]
            elif lower_bound_dev < std_dev < upper_bound_dev:
                score += score_band[1]
            else:
                score += score_band[2]
   #         print("score after std dev: ", score)
        # Add condition based on technical analysis signal
        if rsi_signal == 1:
            score += 5
        elif rsi_signal == 0:
            score += 10
        elif rsi_signal == -1:
            score += 15
  #      print("score after rsi signal: ",score)
        # Condition based on MACD signal
        if macd_signal == 1:
            score += 5
        elif macd_signal == 0:
            score += 10
        elif macd_signal == -1:
            score += 15
 #       print("score after macd: ",score)
    else:
        print("Ticker code not found for sector:", sector_name)
    return score

# CATEGORIZING RISK APPETITE OF USER BASED ON USER INPUT

In [74]:
# Define the categorize_risk_appetite function
def categorize_risk_appetite(investment_objective, source_of_income, time_horizon):
    # Assign scores based on user's answers
    objective_score = 0
    income_score = 0
    time_horizon_score = 0
    
    # Assign scores based on investment objective
    if investment_objective.lower() == 'retirement savings':
        objective_score += 3
    elif investment_objective.lower() == 'wealth preservation':
        objective_score += 1
    elif investment_objective.lower() == 'wealth accumulation':
        objective_score += 2
    
    # Assign scores based on source of income
    if source_of_income.lower() == 'salary':
        income_score += 3
    elif source_of_income.lower() == 'debt':
        income_score += 2
    elif source_of_income.lower() == 'previous investment':
        income_score += 1
    
    # Assign scores based on expected return
    if time_horizon.lower() == 'more than 10 years':
        time_horizon_score += 3
    elif time_horizon.lower() == 'between 5-10 years':
        time_horizon_score += 2
    elif time_horizon.lower() == 'less than 5 years':
        time_horizon_score += 1
    
    # Calculate total score
    total_score = objective_score + income_score + time_horizon_score
    # Categorize risk appetite based on total score
    if total_score >= 7:
        return "High Risk Appetite"
    elif total_score < 4:
        return "Low Risk Appetite"
    else:
        return "Medium Risk Appetite"


# CREATING DIFFERENT RISK APPETITE PORTFOLIOS

In [84]:

def low_risk_portfolio(risk_scores_df):
    # Filter stocks with risk score in low risk range
    low_risk_stocks = risk_scores_df[(risk_scores_df['Risk Score'] < 180)]
    # Sort stocks by risk score (descending order to prioritize higher risk scores)
    low_risk_stocks = low_risk_stocks.sort_values(by='Risk Score', ascending=False)
    # Calculate total inverse risk score
    total_inverse_risk_score = 1 / low_risk_stocks['Risk Score'].sum()
    # Calculate weight of each stock based on inverse risk score
    low_risk_stocks['Weight'] = (1 / low_risk_stocks['Risk Score']) * total_inverse_risk_score
    # Verify that weights sum up to 1
    total_weight = low_risk_stocks['Weight'].sum()
    if not np.isclose(total_weight, 1.0):
        # If weights don't sum up to 1, normalize them
        low_risk_stocks['Weight'] /= total_weight
    return low_risk_stocks


def medium_risk_portfolio(stocks_df):
    # Filter stocks with risk score in medium risk range
    medium_risk_stocks =risk_scores_df[(risk_scores_df['Risk Score'] >= 180) & (risk_scores_df['Risk Score'] < 220)]
    # Sort stocks by market cap
    medium_risk_stocks = medium_risk_stocks.sort_values(by='Risk Score', ascending=True)
    # Calculate weight of each stock
    total_inverse_risk_score = 1 / medium_risk_stocks['Risk Score'].sum()
    medium_risk_stocks['Weight'] = (1 / medium_risk_stocks['Risk Score']) / total_inverse_risk_score
    total_weight = medium_risk_stocks['Weight'].sum()
    if not np.isclose(total_weight, 1.0):
        # If weights don't sum up to 1, normalize them
        medium_risk_stocks['Weight'] /= total_weight
    return medium_risk_stocks

def high_risk_portfolio(stocks_df):
    # Filter stocks with risk score in high risk range
    high_risk_stocks = stocks_df[stocks_df['Risk Score'] >= 225]
    # Sort stocks by risk score (ascending order to prioritize lower risk scores)
    high_risk_stocks = high_risk_stocks.sort_values(by='Risk Score', ascending=True)
    # Calculate total inverse risk score
    total_inverse_risk_score = 1 / high_risk_stocks['Risk Score'].sum()
    # Calculate weight of each stock based on inverse risk score
    high_risk_stocks['Weight'] = (1 / high_risk_stocks['Risk Score']) * total_inverse_risk_score
    # Verify that weights sum up to 1
    total_weight = high_risk_stocks['Weight'].sum()
    if not np.isclose(total_weight, 1.0):
        # If weights don't sum up to 1, normalize them
        high_risk_stocks['Weight'] /= total_weight
    return high_risk_stocks


# CALLING INDIVIDUAL RISK SPECIFIC PORTFOLIOS

In [30]:
#Creating Portfolio 
def portfolio(risk_scores_df, user_risk):
    # Call individual risk-specific portfolio functions
    
    low_risk_portfolio_df = low_risk_portfolio(risk_scores_df)
   
    medium_risk_portfolio_df = medium_risk_portfolio(risk_scores_df)
 
    high_risk_portfolio_df = high_risk_portfolio(risk_scores_df)
    
    if user_risk=="Low Risk Appetite": 
        return low_risk_portfolio_df
    elif user_risk=="Medium Risk Appetite":
        return medium_risk_portfolio_df   
    else:
        return high_risk_portfolio_df

In [72]:

# Define calculate_portfolio_return function
def calculate_portfolio_return(portfolio_df):
    portfolio_df['Weighted Return'] = portfolio_df['Annualized Return'] * portfolio_df['Weight']
    portfolio_return = portfolio_df['Weighted Return'].sum()
    return portfolio_return

# Define calculate_portfolio_risk function
def calculate_portfolio_risk(portfolio_df):
    portfolio_df['Weighted Risk'] = portfolio_df['Annualized Risk'] * portfolio_df['Weight']
    portfolio_risk = portfolio_df['Weighted Risk'].sum()
    return portfolio_risk

#Define calculate portfolio_beta function
def calculate_portfolio_beta(portfolio_df,Beta_df):
    # Merge portfolio_df with beta_df on 'Company' column
    merged_df = pd.merge(portfolio_df, Beta_df, on='Company', how='left')
    
    # Calculate weighted beta for each asset in the portfolio
    merged_df['Weighted Beta'] = merged_df['Weight'] * merged_df['Beta']
    
    # Calculate portfolio beta
    portfolio_beta = merged_df['Weighted Beta'].sum()
    
    return portfolio_beta

    
# Define risk_return function
def risk_return(portfolio_df,Beta_df):
    # Iterate through each row in portfolio_df
    for index, row in portfolio_df.iterrows():
        ticker = row['Company']
        
        # Fetch historical prices for the current stock
        _, df_historical_prices, _, _ = fetching_data(ticker)
        
        # Calculate log daily returns
        #df_historical_prices['Log Returns'] = np.log(df_historical_prices['Close'] / df_historical_prices['Close'].shift(1))
        
        # Calculate mean of log returns as expected return
        Change_in_return = df_historical_prices['Close'].pct_change()
        
        expected_return=Change_in_return.mean()
        
        # Annualize the expected return
        annualized_return = expected_return *252 # Assuming 252 trading days in a year
        
        # Calculate standard deviation of annualized returns as risk
        annualized_risk = Change_in_return.std() * np.sqrt(252) # Annualized volatility
        
        # Update portfolio_df with calculated values
        portfolio_df.loc[index, 'Expected Return'] = expected_return
        portfolio_df.loc[index, 'Annualized Return'] = annualized_return
        portfolio_df.loc[index, 'Annualized Risk'] = annualized_risk
    
    # Calculate portfolio overall return
    portfolio_return = calculate_portfolio_return(portfolio_df)

    # Calculate portfolio risk (standard deviation)
    portfolio_risk = calculate_portfolio_risk(portfolio_df)
    portfolio_beta = calculate_portfolio_beta(portfolio_df,Beta_df)
    
    risk_free_rate=0.0717
    sharpe_ratio= (portfolio_return-risk_free_rate)/ portfolio_risk #The portfolio risk is the standard deviation of the portfolio
    treynor_ratio= (portfolio_return-risk_free_rate)/ portfolio_beta
    
    return portfolio_return, portfolio_risk, sharpe_ratio, treynor_ratio
